In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import cross_validation
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.5)

import matplotlib.pyplot as plt
# this allows plots to appear directly in the notebook
%matplotlib inline

import sklearn.linear_model

In [ ]:
df = pd.read_csv("yellow_tripdata_2015-01-1p.csv")
df.head()

In [ ]:
df['log_total_amount'] = df['total_amount'].apply(lambda x: np.log(x))

df.head()

In [ ]:
df['total_amount'].hist()

In [ ]:
df['log_total_amount'].hist()

In [ ]:
print df.dtypes

In [ ]:
df.describe()

In [ ]:
def get_dt(x):
    return datetime.strptime(x, '%m/%d/%Y %H:%M')

df['tpep_pickup_datetime'] = df.apply(lambda x: get_dt(x[1]), axis=1)
df['tpep_dropoff_datetime'] = df.apply(lambda x: get_dt(x[2]), axis=1)

df['tpep_pickup_dayofweek'] = df['tpep_pickup_datetime'].dt.dayofweek
#df['tpep_dropoff_dayofweek'] = df['tpep_dropoff_datetime'].dt.dayofweek

#df['tpep_pickup_time'] = df['tpep_pickup_time'].astype("int")
df['tpep_pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

#df['tpep_pickup_minuteofday'] = df['tpep_pickup_time'].apply(lambda x: x.hour) * 60 + df['tpep_pickup_time'].apply(lambda x: x.minute)
#df['tpep_dropoff_time'] = df['tpep_dropoff_datetime'].dt.time

In [ ]:
df = df.drop(['store_and_fwd_flag',
              'pickup_longitude',
              'pickup_latitude',
              'tpep_pickup_datetime', 
              'tpep_dropoff_datetime',
              'dropoff_longitude',
              'dropoff_latitude'], axis=1)

In [ ]:
df = df[df['RateCodeID'] == 1]
df = df[df['borough'] == 'Manhattan']
df = df[df['passenger_count']!= 0]
df = df[df['trip_distance']!= 0]
df = df[df['fare_amount'] > 0]
df = df[df['borough']!= '0']
df.head()

In [ ]:
g = sns.pairplot(df, vars=['total_amount', 'tpep_pickup_dayofweek', 'tpep_pickup_hour', 'tip_amount'], hue="neighborhood")

In [ ]:
# visualize the relationship between the features and the response using scatterplots
df.plot(kind ='scatter', x='tpep_pickup_hour', y='total_amount', alpha=0.1)
plt.xlabel('Hour')
plt.ylabel('Total Amount')
plt.axis([0, 24, 0, 100])
plt.grid(True)
plt.show()

In [ ]:
# visualize the relationship between the features and the response using scatterplots
df.plot(kind ='scatter', x='tpep_pickup_dayofweek', y='total_amount', alpha=0.1)
plt.xlabel('Day of Week')
plt.ylabel('Total Amount')
plt.axis([-1, 7, 0, 100])
plt.grid(True)
plt.show()

In [ ]:
sns.boxplot(x='tpep_pickup_dayofweek', y='total_amount', data=df)
plt.ylim(0, 150)

In [ ]:
sns.boxplot(x='tpep_pickup_hour', y='total_amount', data=df)
plt.ylim(0, 150)

In [ ]:
df = df.join(pd.get_dummies(df['tpep_pickup_hour'], prefix='hour'))
df = df.join(pd.get_dummies(df['tpep_pickup_dayofweek'], prefix='week'))
df = df.join(pd.get_dummies(df['neighborhood'], prefix='n'))
#df = pd.get_dummies(df).astype(np.int8)
df.head()

In [ ]:
df = df.drop(['VendorID',
              'neighborhood',
              'RateCodeID',
              'tpep_pickup_dayofweek',
              'tpep_pickup_hour',
              'borough'], axis=1)

In [ ]:
df.dtypes

In [ ]:
f, ax = plt.subplots(figsize=(15, 15))
cmap = sns.diverging_palette(300, 10, as_cmap=True)

correlations = df.corr()
print correlations
print sns.heatmap(correlations, cmap=cmap, ax=ax)

In [ ]:
import statsmodels.formula.api as smf

y = df['total_amount']
log_y = np.log10(y+1)
lm = smf.ols(formula=' log_y ~ hour_0 + hour_1 + hour_2 + hour_3 + hour_4 + hour_5 + hour_6 + hour_7 + hour_8 + hour_9 + hour_10 + hour_11 + hour_12 + hour_13 + hour_14 + hour_15 + hour_16 + hour_18 + hour_19 + hour_20 + hour_21 + hour_22 + hour_23 + week_0 + week_1 + week_2 + week_3 + week_4 + week_5 + week_6' , data=df).fit()
#print the full summary
lm.summary()

In [ ]:
# Set target variable name
target = 'total_amount'

# Set X and y
X = df.drop([target], axis=1)
y = df[target]


# Create separate training and test sets with 60/40 train/test split
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.4, random_state=0)


# check size of training set
print X_train.shape, y_train.shape

# check size of test set
print X_test.shape, y_test.shape

In [ ]:
from sklearn import feature_selection, linear_model

def get_linear_model_metrics(X, y, algo):
    # get the pvalue of X given y. Ignore f-stat for now.
    pvals = feature_selection.f_regression(X, y)[1]
    # start with an empty linear regression object
    # .fit() runs the linear regression function on X and y
    algo.fit(X,y)
    residuals = (y-algo.predict(X)).values

    # print the necessary values
    print 'P Values:', pvals
    print 'Coefficients:', algo.coef_
    print 'y-intercept:', algo.intercept_
    print 'R-Squared:', algo.score(X,y)
    #plot hist of errors
    plt.figure()
    plt.hist(residuals, bins=np.ceil(np.sqrt(len(y))))
    # keep the model
    return algo

lm = linear_model.LinearRegression(fit_intercept=False)
lm = get_linear_model_metrics(X_train, y_train, lm)

In [ ]:
kf = cross_validation.KFold(len(X_train), n_folds=5, shuffle=True)

In [ ]:
from sklearn import metrics

mse_values = []
scores = []
n= 0
print "~~~~ CROSS VALIDATION each fold ~~~~"
for train_index, test_index in kf:
    lm = linear_model.LinearRegression().fit(X_train.iloc[train_index], y_train.iloc[train_index])
    mse_values.append(metrics.mean_squared_error(y_train.iloc[test_index], lm.predict(X_train.iloc[test_index])))
    scores.append(lm.score(X_train, y_train))
    n+=1
    print 'Model', n
    print 'MSE:', mse_values[n-1]
    print 'R2:', scores[n-1]


print "~~~~ SUMMARY OF CROSS VALIDATION ~~~~"
print 'Mean of MSE for all folds:', np.mean(mse_values)
print 'Mean of R2 for all folds:', np.mean(scores)

In [ ]:
lm = linear_model.LinearRegression().fit(X_train, y_train)
print "~~~~ Single Model ~~~~"
print 'MSE of single model:', metrics.mean_squared_error(y_train, lm.predict(X_train))
print 'R2: ', lm.score(X_train, y_train)

In [ ]:
# fit linear regression using no regularization (OLS)
lm = linear_model.LinearRegression().fit(X_train, y_train)
print "~~~ No regularization (OLS) ~~~"
print 'OLS MSE: ', metrics.mean_squared_error(y_train, lm.predict(X_train))
print 'OLS R2:', lm.score(X_train, y_train)

# fit linear regression using L1 regularization (Lasso)
lm = linear_model.Lasso().fit(X_train, y_train)
print "~~~ L1 regularization (Lasso) ~~~"
print 'Lasso MSE: ', metrics.mean_squared_error(y_train, lm.predict(X_train))
print 'Lasso R2:', lm.score(X_train, y_train)

# fit linear regression using L2 regularization (Ridge)
lm = linear_model.Ridge().fit(X_train, y_train)
print "~~~ L2 regularization (Ridge) ~~~"
print 'Ridge MSE: ', metrics.mean_squared_error(y_train, lm.predict(X_train))
print 'Ridge R2:', lm.score(X_train, y_train)

In [ ]:
alphas = np.logspace(-10, 10, 21)
for a in alphas:
    print 'Alpha:', a
    lm = linear_model.Ridge(alpha=a)
    lm.fit(X_train, y_train)
    print metrics.mean_squared_error(y_train, lm.predict(X_train))

In [ ]:
# import grid search
from sklearn import grid_search

# pick range of values to search with
alphas = np.logspace(-10, 10, 21)

# use grid search CV to find best value
gs = grid_search.GridSearchCV(
    estimator=linear_model.Ridge(),
    param_grid={'alpha': alphas},
    scoring='mean_squared_error')
gs.fit(X_train, y_train)

In [ ]:
print gs.best_estimator_

In [ ]:
print gs.grid_scores_

In [ ]:
num_to_approach, start, steps, optimized = 6.2, 0., [-1, 1], False
while not optimized:
    current_distance = num_to_approach - start
    got_better = False
    next_steps = [start + i for i in steps]
    for n in next_steps:
        distance = np.abs(num_to_approach - n)
        if distance < current_distance:
            got_better = True
            print distance, 'is better than', current_distance
            current_distance = distance
            start = n
    if got_better:
        print 'found better solution! using', current_distance
        a += 1
    else:
        optimized = True
        print start, 'is closest to', num_to_approach

In [ ]:
num_to_approach, start, steps, optimized = 6.2, 0., [-1, 1], False
n_iter = 0
while not optimized:
    if n_iter > 3:
        print 'stopping iterations'
        break
    n_iter += 1
    current_distance = num_to_approach - start
    got_better = False
    next_steps = [start + i for i in steps]
    for n in next_steps:
        distance = np.abs(num_to_approach - n)
        if distance < current_distance:
            got_better = True
            print distance, 'is better than', current_distance
            current_distance = distance
            start = n
    if got_better:
        print 'found better solution! using', current_distance
        a += 1
    else:
        optimized = True
        print start, 'is closest to', num_to_approach

In [ ]:
lm = linear_model.SGDRegressor()
lm.fit(X_train, y_train)
print "Gradient Descent MSE:", metrics.mean_squared_error(y_train, lm.predict(X_train))
print "Gradient Descent R2:", lm.score(X_train, y_train)

In [ ]:
lm = linear_model.SGDRegressor()
lm.fit(X_test, y_test)
print "Gradient Descent MSE:", metrics.mean_squared_error(y_test, lm.predict(X_test))
print "Gradient Descent R2:", lm.score(X_test, y_test)